## Alternate lag testing ##

Here we test a new methodology for the alternate lagging. Having calculated the anomalies for all of the models, ensemble members, start dates, and forecast years in the '*calc_anoms_suite*', we want to load these into python to form arrays with shapes like:

(178, 60, 11, 72, 144)

Where the dimensions are as following:

* 178 total ensemble members (from all of the models)
* 60 start dates (~1960-2020)
* 11 forecast years (could this differ between models - *may have to watch out for*)
* 72 latitude bands
* 144 longitude bands

As a first exercise, it would be useful to load in an array for a single model, in this case: BCC-CSM2-MR. The shape would look something like:

(8, 60, 11, 72, 144)

In [1]:
# Import local modules
import sys
import os
import argparse

# Import 3rd party modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr

In [2]:
# Set up the arguments
base_dir = "/gws/nopw/j04/canari/users/benhutch/skill-maps-processed-data"
variable = "psl"
model = "BCC-CSM2-MR"
region = "global"
forecast_range = "all_forecast_years"
season = "DJFM"

In [4]:
# Form the directory path
dir_path = os.path.join(base_dir, variable, model, region, forecast_range,
                        season, "outputs", "anoms")

print(dir_path)

# List the files ending with *.nc in this directory
file_list = [f for f in os.listdir(dir_path) if f.endswith(".nc")]

# Print the list of files
print(file_list)

/gws/nopw/j04/canari/users/benhutch/skill-maps-processed-data/psl/BCC-CSM2-MR/global/all_forecast_years/DJFM/outputs/anoms
['all-years-DJFM-global-psl_Amon_BCC-CSM2-MR_dcppA-hindcast_s1970-r1i1p1f1_gn_197001-197912-anoms.nc', 'all-years-DJFM-global-psl_Amon_BCC-CSM2-MR_dcppA-hindcast_s1970-r2i1p1f1_gn_197001-197912-anoms.nc', 'all-years-DJFM-global-psl_Amon_BCC-CSM2-MR_dcppA-hindcast_s1970-r3i1p1f1_gn_197001-197912-anoms.nc', 'all-years-DJFM-global-psl_Amon_BCC-CSM2-MR_dcppA-hindcast_s1970-r4i1p1f1_gn_197001-197912-anoms.nc', 'all-years-DJFM-global-psl_Amon_BCC-CSM2-MR_dcppA-hindcast_s1970-r5i1p1f1_gn_197001-197912-anoms.nc', 'all-years-DJFM-global-psl_Amon_BCC-CSM2-MR_dcppA-hindcast_s1970-r6i1p1f1_gn_197001-197912-anoms.nc', 'all-years-DJFM-global-psl_Amon_BCC-CSM2-MR_dcppA-hindcast_s1970-r7i1p1f1_gn_197001-197912-anoms.nc', 'all-years-DJFM-global-psl_Amon_BCC-CSM2-MR_dcppA-hindcast_s1970-r8i1p1f1_gn_197001-197912-anoms.nc', 'all-years-DJFM-global-psl_Amon_BCC-CSM2-MR_dcppA-hindcast_s